# Phase 4: Calculate Raw Components

This notebook calculates the three raw components for the effectiveness scoring methodology:
1. Component 1: Severity Improvement Rate
2. Component 2: Consistency Score  
3. Component 3: Cost-Effectiveness

**Note**: Using `inform_severity_with_funding.csv` which includes real funding data from HRP datasets.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Load the standardized INFORM severity data with funding
df = pd.read_csv("inform_severity_with_funding.csv")

print(f"Loaded {len(df)} rows")
print(f"Unique crises: {df['CRISIS ID'].nunique()}")
print(f"Date range: {df['year'].min()}-{df['year'].max()}")
print(f"Funding data available: {df['Total_Funding'].notna().sum()} / {len(df)} rows ({df['Total_Funding'].notna().sum()/len(df)*100:.1f}%)")

Loaded 5292 rows
Unique crises: 157
Date range: 2020-unknown_year


In [4]:
# Create date column (reuse logic from Phase 1)
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4,
    'may': 5, 'june': 6, 'july': 7, 'august': 8,
    'september': 9, 'october': 10, 'november': 11, 'december': 12
}

df['month_clean'] = df['month'].astype(str).str.lower().str.strip()
df['month_clean'] = df['month_clean'].replace({
    'inform_severity_mid_december': 'december',
    'late_november': 'november'
})
df['month_num'] = df['month_clean'].map(month_map)
df['year_clean'] = pd.to_numeric(df['year'], errors='coerce')

df['date'] = pd.NaT
valid_mask = df['month_num'].notna() & df['year_clean'].notna()
df.loc[valid_mask, 'date'] = pd.to_datetime(
    {
        'year': df.loc[valid_mask, 'year_clean'],
        'month': df.loc[valid_mask, 'month_num'],
        'day': 1
    },
    errors='coerce'
)

# Sort by crisis and date
df = df.sort_values(['CRISIS ID', 'date'])

print(f"Valid dates: {df['date'].notna().sum()} / {len(df)}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

Valid dates: 5208 / 5292
Date range: 2020-09-01 00:00:00 to 2025-12-01 00:00:00


## Step 4.1: Component 1 - Raw Improvement Rate (Point-in-Time)

Calculate: `Improvement_Rate_Raw = (INFORM_Start - INFORM_Current) / Duration_Months`

**IMPORTANT**: Using point-in-time approach - calculates score at each month using only PAST data (no future information).

Where:
- INFORM_Start = First valid INFORM Severity Index for the crisis
- INFORM_Current = Current month's INFORM Severity Index  
- Duration_Months = Number of months from start to current point

In [ ]:
# Point-in-time calculation: Calculate improvement rate at each month
# using only data available UP TO that point (no future information)

crisis_summary = []

for crisis_id in df['CRISIS ID'].unique():
    crisis_data = df[df['CRISIS ID'] == crisis_id].copy()
    
    # Filter to rows with valid dates and severity values
    valid_data = crisis_data[
        crisis_data['date'].notna() & 
        crisis_data['INFORM Severity Index'].notna()
    ].sort_values('date')
    
    if len(valid_data) < 2:
        # Need at least 2 observations to calculate improvement
        continue
    
    # Get the start value (first observation)
    inform_start = valid_data.iloc[0]['INFORM Severity Index']
    date_start = valid_data.iloc[0]['date']
    
    # Calculate improvement rate at each point in time
    # Store the final month's score (using only data up to that point)
    for i in range(1, len(valid_data)):
        # Use only data up to current point
        historical_data = valid_data.iloc[:i+1]
        
        inform_current = historical_data.iloc[-1]['INFORM Severity Index']
        date_current = historical_data.iloc[-1]['date']
        
        # Calculate duration in months
        duration_months = (date_current.year - date_start.year) * 12 + (date_current.month - date_start.month)
        if duration_months == 0:
            duration_months = 1
        
        # Calculate improvement rate (positive = improved, negative = worsened)
        improvement_rate_raw = (inform_start - inform_current) / duration_months
        
        # Store the final month's calculation
        if i == len(valid_data) - 1:
            crisis_summary.append({
                'CRISIS ID': crisis_id,
                'COUNTRY': valid_data.iloc[0]['COUNTRY'],
                'ISO3': valid_data.iloc[0]['ISO3'],
                'INFORM_Start': inform_start,
                'INFORM_End': inform_current,  # Final value using only past data
                'Date_Start': date_start,
                'Date_End': date_current,
                'Duration_Months': duration_months,
                'N_Observations': len(historical_data),
                'Improvement_Rate_Raw': improvement_rate_raw
            })

hrp_data = pd.DataFrame(crisis_summary)

print(f"\nCalculated improvement rates for {len(hrp_data)} crises (point-in-time)")
print(f"\n=== RAW IMPROVEMENT RATE STATISTICS ===")
print(f"Mean: {hrp_data['Improvement_Rate_Raw'].mean():.4f}")
print(f"Std Dev: {hrp_data['Improvement_Rate_Raw'].std():.4f}")
print(f"Min (worst): {hrp_data['Improvement_Rate_Raw'].min():.4f}")
print(f"Max (best): {hrp_data['Improvement_Rate_Raw'].max():.4f}")
print(f"Median: {hrp_data['Improvement_Rate_Raw'].median():.4f}")

# How many improved vs worsened?
improved = (hrp_data['Improvement_Rate_Raw'] > 0).sum()
worsened = (hrp_data['Improvement_Rate_Raw'] < 0).sum()
unchanged = (hrp_data['Improvement_Rate_Raw'] == 0).sum()

print(f"\nImproved: {improved} ({improved/len(hrp_data)*100:.1f}%)")
print(f"Worsened: {worsened} ({worsened/len(hrp_data)*100:.1f}%)")
print(f"Unchanged: {unchanged}")

# Display sample
print(f"\nSample crises:")
print(hrp_data[['CRISIS ID', 'COUNTRY', 'INFORM_Start', 'INFORM_End', 'Duration_Months', 'Improvement_Rate_Raw']].head(10))


Calculated improvement rates for 149 crises

=== RAW IMPROVEMENT RATE STATISTICS ===
Mean: -0.0021
Std Dev: 0.0233
Min (worst): -0.1000
Max (best): 0.1000
Median: 0.0000

Improved: 47 (31.5%)
Worsened: 70 (47.0%)
Unchanged: 32

Sample crises:
  CRISIS ID       COUNTRY  INFORM_Start  INFORM_End  Duration_Months  \
0    AFG001   Afghanistan           4.6         4.5               63   
1    AGO002        Angola           3.2         3.2               48   
2    ARM002       Armenia           1.7         2.1               45   
3    ARM003       Armenia           2.2         2.1                4   
4    AZE002    Azerbaijan           1.8         1.6               27   
5    BDI001       Burundi           3.3         3.2               58   
6    BDI005       Burundi           3.4         3.4                4   
7    BEN002         Benin           1.9         2.2               19   
8    BFA002  Burkina Faso           3.9         4.0               63   
9    BFA004  Burkina Faso           

In [ ]:
# Calculate consistency using point-in-time approach
# Standard deviation calculated using only historical data up to final month

consistency_data = []

for crisis_id in hrp_data['CRISIS ID']:
    crisis_data = df[df['CRISIS ID'] == crisis_id].copy()
    
    # Get the final date for this crisis
    final_date = hrp_data[hrp_data['CRISIS ID'] == crisis_id]['Date_End'].iloc[0]
    
    # Filter to rows with valid severity values UP TO final date
    valid_severity = crisis_data[
        (crisis_data['date'].notna()) & 
        (crisis_data['date'] <= final_date) &
        (crisis_data['INFORM Severity Index'].notna())
    ]['INFORM Severity Index']
    
    if len(valid_severity) < 2:
        # Need at least 2 observations for std dev
        inform_std = np.nan
    else:
        # Calculate std dev using only historical data
        inform_std = valid_severity.std()
    
    consistency_data.append({
        'CRISIS ID': crisis_id,
        'INFORM_Std': inform_std
    })

consistency_df = pd.DataFrame(consistency_data)
hrp_data = hrp_data.merge(consistency_df, on='CRISIS ID', how='left')

# Calculate consistency raw score
# Lower std = higher consistency (we invert)
hrp_data['Consistency_Raw'] = 1 / (1 + hrp_data['INFORM_Std'].fillna(1))

print(f"\n=== RAW CONSISTENCY STATISTICS (Point-in-Time) ===")
print(f"Mean INFORM_Std: {hrp_data['INFORM_Std'].mean():.4f}")
print(f"Mean Consistency_Raw: {hrp_data['Consistency_Raw'].mean():.4f}")
print(f"Std Dev Consistency_Raw: {hrp_data['Consistency_Raw'].std():.4f}")
print(f"Min Consistency_Raw: {hrp_data['Consistency_Raw'].min():.4f}")
print(f"Max Consistency_Raw: {hrp_data['Consistency_Raw'].max():.4f}")

# Show distribution
print(f"\nConsistency score interpretation:")
print(f"  Perfectly flat (std=0): {1/(1+0):.4f}")
print(f"  Low volatility (std=0.1): {1/(1+0.1):.4f}")
print(f"  Moderate volatility (std=1.0): {1/(1+1.0):.4f}")
print(f"  High volatility (std=4.0): {1/(1+4.0):.4f}")

print(f"\nSample crises with consistency scores:")
sample_cols = ['CRISIS ID', 'COUNTRY', 'INFORM_Std', 'Consistency_Raw']
print(hrp_data[sample_cols].head(10))


=== RAW CONSISTENCY STATISTICS ===
Mean INFORM_Std: 0.1581
Mean Consistency_Raw: 0.8707
Std Dev Consistency_Raw: 0.0779
Min Consistency_Raw: 0.6748
Max Consistency_Raw: 1.0000

Consistency score interpretation:
  Perfectly flat (std=0): 1.0000
  Low volatility (std=0.1): 0.9091
  Moderate volatility (std=1.0): 0.5000
  High volatility (std=4.0): 0.2000

Sample crises with consistency scores:
  CRISIS ID       COUNTRY  INFORM_Std  Consistency_Raw
0    AFG001   Afghanistan    0.099391         0.909595
1    AGO002        Angola    0.155921         0.865111
2    ARM002       Armenia    0.372153         0.728782
3    ARM003       Armenia    0.044721         0.957193
4    AZE002    Azerbaijan    0.241112         0.805729
5    BDI001       Burundi    0.220541         0.819309
6    BDI005       Burundi    0.044721         0.957193
7    BEN002         Benin    0.196529         0.835751
8    BFA002  Burkina Faso    0.140802         0.876576
9    BFA004  Burkina Faso    0.057735         0.945416

In [8]:
consistency_df.head()

,CRISIS ID,INFORM_Std
0,AFG001,0.099391
1,AGO002,0.155921
2,ARM002,0.372153
3,ARM003,0.044721
4,AZE002,0.241112


## Step 4.3: Component 3 - Raw Cost-Effectiveness

Calculate: `Cost_Effectiveness_Raw = Improvement_Rate_Raw / log(Funding_Per_Month + 1)`

**Note**: Funding data is not in the INFORM severity CSV. We need to:
1. Join funding data from HRP (Humanitarian Response Plan) datasets, OR
2. Use placeholder values for now

For now, we'll create a placeholder structure and note where funding should be joined.

In [ ]:
# TODO: Join funding data from HRP datasets
# Funding should come from humanitarian response plan data
# For now, we'll create placeholder values to show the calculation structure

print("="*60)
print("FUNDING DATA PLACEHOLDER")
print("="*60)
print("Funding data needs to be joined from HRP datasets.")
print("Expected columns: Funding_Requested (total USD)")
print("="*60)

# Create placeholder funding (you'll replace this with actual HRP join)
# Using a reasonable placeholder: $50M average per crisis
np.random.seed(42)
hrp_data['Funding_Requested'] = np.random.lognormal(mean=17, sigma=1, size=len(hrp_data))
hrp_data['Funding_Requested'] = hrp_data['Funding_Requested'].round(0)

# Calculate funding per month
hrp_data['Funding_Per_Month'] = hrp_data['Funding_Requested'] / hrp_data['Duration_Months']

# Calculate cost-effectiveness
# Improvement per log-dollar (log dampens effect of large funding differences)
hrp_data['Cost_Effectiveness_Raw'] = hrp_data['Improvement_Rate_Raw'] / np.log(hrp_data['Funding_Per_Month'] + 1)

# Handle edge cases
hrp_data['Cost_Effectiveness_Raw'] = hrp_data['Cost_Effectiveness_Raw'].replace([np.inf, -np.inf], np.nan)

print(f"\n=== RAW COST-EFFECTIVENESS STATISTICS ===")
print(f"Mean: {hrp_data['Cost_Effectiveness_Raw'].mean():.4f}")
print(f"Std Dev: {hrp_data['Cost_Effectiveness_Raw'].std():.4f}")
print(f"Min: {hrp_data['Cost_Effectiveness_Raw'].min():.4f}")
print(f"Max: {hrp_data['Cost_Effectiveness_Raw'].max():.4f}")
print(f"Missing values: {hrp_data['Cost_Effectiveness_Raw'].isna().sum()}")

# Fill any NaN with 0 (neutral)
hrp_data['Cost_Effectiveness_Raw'] = hrp_data['Cost_Effectiveness_Raw'].fillna(0)

print(f"\nSample cost-effectiveness calculations:")
sample_cols = ['CRISIS ID', 'COUNTRY', 'Funding_Requested', 'Funding_Per_Month', 
                'Improvement_Rate_Raw', 'Cost_Effectiveness_Raw']
print(hrp_data[sample_cols].head(10))

## Summary: All Raw Components Calculated

All three raw components are now calculated:
1. ✅ **Improvement_Rate_Raw**: Severity change per month
2. ✅ **Consistency_Raw**: Inverse of volatility (0-1 scale)
3. ✅ **Cost_Effectiveness_Raw**: Improvement per log-dollar

**Next Steps:**
- Replace placeholder funding with actual HRP data join
- Normalize all components to 0-1 scale (Phase 5)
- Calculate composite effectiveness score

In [ ]:
# Save the results
hrp_data.to_csv('hrp_data_with_raw_components.csv', index=False)
print("Saved results to: hrp_data_with_raw_components.csv")

# Display final summary
print("\n" + "="*60)
print("FINAL SUMMARY")
print("="*60)
print(f"Total crises analyzed: {len(hrp_data)}")
print(f"\nComponent 1 - Improvement Rate:")
print(f"  Range: [{hrp_data['Improvement_Rate_Raw'].min():.4f}, {hrp_data['Improvement_Rate_Raw'].max():.4f}]")
print(f"\nComponent 2 - Consistency:")
print(f"  Range: [{hrp_data['Consistency_Raw'].min():.4f}, {hrp_data['Consistency_Raw'].max():.4f}]")
print(f"\nComponent 3 - Cost-Effectiveness:")
print(f"  Range: [{hrp_data['Cost_Effectiveness_Raw'].min():.4f}, {hrp_data['Cost_Effectiveness_Raw'].max():.4f}]")
print("="*60)

# Phase 5: Normalize to 0-1 Scale

Normalize all three raw components to 0-1 scale before combining them into the composite effectiveness score.

In [ ]:
# Point-in-time normalization: Normalize using only historical data
# For each crisis, use min/max from crises that ended before or at the same time

print("Normalizing components using point-in-time approach...")
print("Each crisis normalized using min/max from crises ending before or at same time.\n")

## Step 5.1: Normalize Improvement Rate

In [ ]:
# Normalize Improvement Rate using point-in-time approach
# Sort by end date to process chronologically
hrp_data_sorted = hrp_data.sort_values('Date_End').reset_index(drop=True)

improvement_normalized = []
for idx, row in hrp_data_sorted.iterrows():
    current_date = row['Date_End']
    current_value = row['Improvement_Rate_Raw']
    
    # Get all crises that ended before or at current date
    historical_data = hrp_data_sorted[hrp_data_sorted['Date_End'] <= current_date]['Improvement_Rate_Raw']
    
    if len(historical_data) == 0:
        normalized_val = 0.5
    else:
        min_val = historical_data.min()
        max_val = historical_data.max()
        
        if max_val == min_val:
            normalized_val = 0.5
        else:
            normalized_val = (current_value - min_val) / (max_val - min_val)
    
    improvement_normalized.append(normalized_val)

hrp_data_sorted['Improvement_Rate_Normalized'] = improvement_normalized
hrp_data = hrp_data_sorted.sort_values('CRISIS ID').reset_index(drop=True)

print("\n=== NORMALIZED IMPROVEMENT RATE (0-1, Point-in-Time) ===")
print(f"Mean: {hrp_data['Improvement_Rate_Normalized'].mean():.4f}")
print(f"Std Dev: {hrp_data['Improvement_Rate_Normalized'].std():.4f}")
print(f"Min: {hrp_data['Improvement_Rate_Normalized'].min():.4f}")
print(f"Max: {hrp_data['Improvement_Rate_Normalized'].max():.4f}")

# Verify bounds
assert hrp_data['Improvement_Rate_Normalized'].min() >= 0, "ERROR: Minimum below 0"
assert hrp_data['Improvement_Rate_Normalized'].max() <= 1, "ERROR: Maximum above 1"
print("Bounds check passed (0-1)")

print("\nInterpretation:")
print("  1.0 = Best improvement among crises ending before/at same time")
print("  0.0 = Worst worsening among crises ending before/at same time")
print("  0.5 = Middle of historical range")

## Step 5.2: Normalize Consistency

In [ ]:
# Normalize Consistency using point-in-time approach
hrp_data_sorted = hrp_data.sort_values('Date_End').reset_index(drop=True)

consistency_normalized = []
for idx, row in hrp_data_sorted.iterrows():
    current_date = row['Date_End']
    current_value = row['Consistency_Raw']
    
    # Get all crises that ended before or at current date
    historical_data = hrp_data_sorted[hrp_data_sorted['Date_End'] <= current_date]['Consistency_Raw']
    
    if len(historical_data) == 0:
        normalized_val = 0.5
    else:
        min_val = historical_data.min()
        max_val = historical_data.max()
        
        if max_val == min_val:
            normalized_val = 0.5
        else:
            normalized_val = (current_value - min_val) / (max_val - min_val)
    
    consistency_normalized.append(normalized_val)

hrp_data_sorted['Consistency_Normalized'] = consistency_normalized
hrp_data = hrp_data_sorted.sort_values('CRISIS ID').reset_index(drop=True)

print("\n=== NORMALIZED CONSISTENCY (0-1, Point-in-Time) ===")
print(f"Mean: {hrp_data['Consistency_Normalized'].mean():.4f}")
print(f"Std Dev: {hrp_data['Consistency_Normalized'].std():.4f}")
print(f"Min: {hrp_data['Consistency_Normalized'].min():.4f}")
print(f"Max: {hrp_data['Consistency_Normalized'].max():.4f}")

# Verify bounds
assert hrp_data['Consistency_Normalized'].min() >= 0, "ERROR: Minimum below 0"
assert hrp_data['Consistency_Normalized'].max() <= 1, "ERROR: Maximum above 1"
print("Bounds check passed (0-1)")

print("\nInterpretation:")
print("  1.0 = Most consistent among crises ending before/at same time")
print("  0.0 = Most volatile among crises ending before/at same time")
print("  0.5 = Middle of historical distribution")

## Step 5.3: Normalize Cost-Effectiveness

In [ ]:
# Normalize Cost-Effectiveness using point-in-time approach
hrp_data_sorted = hrp_data.sort_values('Date_End').reset_index(drop=True)

cost_normalized = []
for idx, row in hrp_data_sorted.iterrows():
    current_date = row['Date_End']
    current_value = row['Cost_Effectiveness_Raw']
    
    # Get all crises that ended before or at current date
    historical_data = hrp_data_sorted[hrp_data_sorted['Date_End'] <= current_date]['Cost_Effectiveness_Raw']
    
    if len(historical_data) == 0:
        normalized_val = 0.5
    else:
        min_val = historical_data.min()
        max_val = historical_data.max()
        
        if max_val == min_val:
            normalized_val = 0.5
        else:
            normalized_val = (current_value - min_val) / (max_val - min_val)
    
    cost_normalized.append(normalized_val)

hrp_data_sorted['Cost_Effectiveness_Normalized'] = cost_normalized
hrp_data = hrp_data_sorted.sort_values('CRISIS ID').reset_index(drop=True)

print("\n=== NORMALIZED COST-EFFECTIVENESS (0-1, Point-in-Time) ===")
print(f"Mean: {hrp_data['Cost_Effectiveness_Normalized'].mean():.4f}")
print(f"Std Dev: {hrp_data['Cost_Effectiveness_Normalized'].std():.4f}")
print(f"Min: {hrp_data['Cost_Effectiveness_Normalized'].min():.4f}")
print(f"Max: {hrp_data['Cost_Effectiveness_Normalized'].max():.4f}")

# Verify bounds
assert hrp_data['Cost_Effectiveness_Normalized'].min() >= 0, "ERROR: Minimum below 0"
assert hrp_data['Cost_Effectiveness_Normalized'].max() <= 1, "ERROR: Maximum above 1"
print("Bounds check passed (0-1)")

print("\nInterpretation:")
print("  1.0 = Best cost-effectiveness among crises ending before/at same time")
print("  0.0 = Worst cost-effectiveness among crises ending before/at same time")
print("  0.5 = Average of historical range")

## Step 5.4: Visualize Component Distributions

In [ ]:
# Create visualization of all three normalized components
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Component 1: Improvement Rate
axes[0].hist(hrp_data['Improvement_Rate_Normalized'], bins=30, 
             edgecolor='black', color='#009EDB', alpha=0.7)
axes[0].axvline(0.5, color='red', linestyle='--', label='Midpoint')
axes[0].set_xlabel('Improvement Rate (Normalized)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Component 1: Improvement Rate\n(0=Worst, 1=Best)', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Component 2: Consistency
axes[1].hist(hrp_data['Consistency_Normalized'], bins=30, 
             edgecolor='black', color='#1A9850', alpha=0.7)
axes[1].axvline(0.5, color='red', linestyle='--', label='Midpoint')
axes[1].set_xlabel('Consistency (Normalized)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Component 2: Consistency\n(0=Volatile, 1=Steady)', fontsize=12)
axes[1].legend()
axes[1].grid(alpha=0.3)

# Component 3: Cost-Effectiveness
axes[2].hist(hrp_data['Cost_Effectiveness_Normalized'], bins=30, 
             edgecolor='black', color='#FDB863', alpha=0.7)
axes[2].axvline(0.5, color='red', linestyle='--', label='Midpoint')
axes[2].set_xlabel('Cost-Effectiveness (Normalized)', fontsize=11)
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title('Component 3: Cost-Effectiveness\n(0=Inefficient, 1=Efficient)', fontsize=12)
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.savefig("person2_component_distributions.png", dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: person2_component_distributions.png")

# Phase 6: Calculate Composite Effectiveness Score

Combine the three normalized components using weighted average to create the final effectiveness score.

## Step 6.1: Apply Weights and Combine

In [ ]:
# Define weights (UPDATED - More punishment for low improvement)
w1 = 0.70  # Improvement Rate (was 0.50) - INCREASED to emphasize actual improvement
w2 = 0.20  # Consistency (was 0.30) - DECREASED
w3 = 0.10  # Cost-Effectiveness (was 0.20) - DECREASED

# Calculate weighted composite score (0-1 scale)
hrp_data['Effectiveness_Score_01'] = (
    w1 * hrp_data['Improvement_Rate_Normalized'] + 
    w2 * hrp_data['Consistency_Normalized'] + 
    w3 * hrp_data['Cost_Effectiveness_Normalized']
)

print("\n=== COMPOSITE EFFECTIVENESS SCORE (0-1 scale) ===")
print(f"Mean: {hrp_data['Effectiveness_Score_01'].mean():.4f}")
print(f"Std Dev: {hrp_data['Effectiveness_Score_01'].std():.4f}")
print(f"Min: {hrp_data['Effectiveness_Score_01'].min():.4f}")
print(f"Max: {hrp_data['Effectiveness_Score_01'].max():.4f}")
print(f"Median: {hrp_data['Effectiveness_Score_01'].median():.4f}")

# Verify score is in 0-1 range
assert hrp_data['Effectiveness_Score_01'].min() >= 0, "ERROR: Score below 0"
assert hrp_data['Effectiveness_Score_01'].max() <= 1, "ERROR: Score above 1"
print("✓ Composite score bounds check passed (0-1)")

# Also create 0-100 version for easier interpretation
hrp_data['Effectiveness_Score_100'] = hrp_data['Effectiveness_Score_01'] * 100

print(f"\n0-100 scale: Mean = {hrp_data['Effectiveness_Score_100'].mean():.1f}")
print(f"            Median = {hrp_data['Effectiveness_Score_100'].median():.1f}")
print(f"            66th percentile = {hrp_data['Effectiveness_Score_100'].quantile(0.66):.1f}")

## Step 6.2: Visualize Effectiveness Score Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(hrp_data['Effectiveness_Score_100'], bins=30, 
             edgecolor='black', color='#009EDB', alpha=0.8)
axes[0].axvline(hrp_data['Effectiveness_Score_100'].median(), 
                color='red', linestyle='--', linewidth=2, label=f"Median ({hrp_data['Effectiveness_Score_100'].median():.1f})")
axes[0].axvline(hrp_data['Effectiveness_Score_100'].quantile(0.66), 
                color='green', linestyle='--', linewidth=2, label=f"66th %ile ({hrp_data['Effectiveness_Score_100'].quantile(0.66):.1f})")
axes[0].set_xlabel('Effectiveness Score (0-100)', fontsize=12)
axes[0].set_ylabel('Number of Crises', fontsize=12)
axes[0].set_title('Distribution of Funding Effectiveness Scores', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(hrp_data['Effectiveness_Score_100'], vert=True)
axes[1].set_ylabel('Effectiveness Score (0-100)', fontsize=12)
axes[1].set_title('Effectiveness Score Distribution\n(Box Plot)', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig("person2_effectiveness_distribution.png", dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: person2_effectiveness_distribution.png")

## Step 6.3: Component Contribution Analysis

In [ ]:
# Calculate how much each component contributes on average
avg_improvement_contribution = w1 * hrp_data['Improvement_Rate_Normalized'].mean()
avg_consistency_contribution = w2 * hrp_data['Consistency_Normalized'].mean()
avg_cost_contribution = w3 * hrp_data['Cost_Effectiveness_Normalized'].mean()

print("\n=== AVERAGE COMPONENT CONTRIBUTIONS TO EFFECTIVENESS ===")
print(f"Improvement Rate contribution: {avg_improvement_contribution:.4f} ({avg_improvement_contribution/hrp_data['Effectiveness_Score_01'].mean()*100:.1f}%)")
print(f"Consistency contribution:       {avg_consistency_contribution:.4f} ({avg_consistency_contribution/hrp_data['Effectiveness_Score_01'].mean()*100:.1f}%)")
print(f"Cost-Effectiveness contribution: {avg_cost_contribution:.4f} ({avg_cost_contribution/hrp_data['Effectiveness_Score_01'].mean()*100:.1f}%)")

# Pie chart of contributions
contributions = [avg_improvement_contribution, avg_consistency_contribution, avg_cost_contribution]
labels = ['Improvement\nRate (50%)', 'Consistency\n(30%)', 'Cost-Effectiveness\n(20%)']
colors = ['#009EDB', '#1A9850', '#FDB863']

plt.figure(figsize=(8, 8))
plt.pie(contributions, labels=labels, autopct='%1.1f%%', 
        colors=colors, startangle=90, textprops={'fontsize': 12})
plt.title('Average Component Contributions to Effectiveness Score', fontsize=14, fontweight='bold')
plt.savefig("person2_component_contributions.png", dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved: person2_component_contributions.png")

## Step 6.4: Identify "Good Crises" (Top Third)

According to the methodology, crises scoring >= 66th percentile are considered "Successful interventions".

In [ ]:
# Calculate 66th percentile threshold
threshold_66 = hrp_data['Effectiveness_Score_100'].quantile(0.66)

# Mark successful interventions
hrp_data['Is_Successful'] = hrp_data['Effectiveness_Score_100'] >= threshold_66

successful_count = hrp_data['Is_Successful'].sum()
total_count = len(hrp_data)

print(f"\n=== 'GOOD CRISIS' IDENTIFICATION ===")
print(f"66th percentile threshold: {threshold_66:.2f}")
print(f"Successful interventions: {successful_count} / {total_count} ({successful_count/total_count*100:.1f}%)")
print(f"\nTop 10 most effective crises:")
top_crises = hrp_data.nlargest(10, 'Effectiveness_Score_100')[
    ['CRISIS ID', 'COUNTRY', 'Effectiveness_Score_100', 
     'Improvement_Rate_Normalized', 'Consistency_Normalized', 'Cost_Effectiveness_Normalized']
]
print(top_crises.to_string(index=False))

# NOTE: Point-in-Time Approach Now Default (Fully Implemented)

**Current Implementation**: All calculations use point-in-time approach:
- ✅ **Improvement Rate**: Calculated using only data up to final month (no future data)
- ✅ **Consistency**: Standard deviation calculated using only historical data up to final month
- ✅ **Cost-Effectiveness**: Uses improvement rate from point-in-time calculation
- ✅ **Normalization**: Uses min/max from only crises ending before/at same time (no future data)

**Result**: Zero data leakage - all scores calculated using only historical information available at the time.

In [ ]:
# Verification: Show that we're using point-in-time approach (fully implemented)
print("="*80)
print("VERIFICATION: Point-in-Time Approach (Fully Implemented)")
print("="*80)
print("\nAll calculations use only historical data:")
print("  ✓ Improvement Rate: Uses data from start to final month (no future)")
print("  ✓ Consistency: Std dev calculated using only data up to final month")
print("  ✓ Cost-Effectiveness: Uses point-in-time improvement rate")
print("  ✓ Normalization: Uses min/max from crises ending before/at same time")
print("\nThis ensures ZERO data leakage - safe for prediction purposes.")
print("="*80)

In [ ]:
# Save final results with all components and scores
hrp_data.to_csv('hrp_data_with_effectiveness_scores.csv', index=False)
print("\n✓ Saved final results to: hrp_data_with_effectiveness_scores.csv")

# Display final summary
print("\n" + "="*60)
print("FINAL SUMMARY - ALL PHASES COMPLETE")
print("="*60)
print(f"Total crises analyzed: {len(hrp_data)}")
print(f"\nRaw Components:")
print(f"  Improvement Rate: [{hrp_data['Improvement_Rate_Raw'].min():.4f}, {hrp_data['Improvement_Rate_Raw'].max():.4f}]")
print(f"  Consistency: [{hrp_data['Consistency_Raw'].min():.4f}, {hrp_data['Consistency_Raw'].max():.4f}]")
print(f"  Cost-Effectiveness: [{hrp_data['Cost_Effectiveness_Raw'].min():.4f}, {hrp_data['Cost_Effectiveness_Raw'].max():.4f}]")
print(f"\nNormalized Components (0-1):")
print(f"  Improvement Rate: Mean = {hrp_data['Improvement_Rate_Normalized'].mean():.4f}")
print(f"  Consistency: Mean = {hrp_data['Consistency_Normalized'].mean():.4f}")
print(f"  Cost-Effectiveness: Mean = {hrp_data['Cost_Effectiveness_Normalized'].mean():.4f}")
print(f"\nComposite Effectiveness Score:")
print(f"  0-1 scale: Mean = {hrp_data['Effectiveness_Score_01'].mean():.4f}")
print(f"  0-100 scale: Mean = {hrp_data['Effectiveness_Score_100'].mean():.1f}")
print(f"  66th percentile threshold: {threshold_66:.1f}")
print(f"  Successful interventions: {successful_count} ({successful_count/total_count*100:.1f}%)")
print("="*60)